## Photo-z Server 
### True redshift catalog example  

Contact author: [Julia Gschwend](mailto:julia@linea.org.br) <br>
Last verified run: **2025-Jan-03**

**Goal**: extract a small sample of true redshifts from DP0.2 using TAP Service on RSP and upload it to the PZ Server as a tutorial example of input for the pipeline *Combine Spec-z Catalogs*. 

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from lsst.rsp import get_tap_service, retrieve_query

Connect to TAP service

In [ ]:
service = get_tap_service('tap')
assert service is not None

Define and execute a query to retrieve the sample with specific constraints. E.g., 1000 relatively bright galaxies in a particular tract: 

In [ ]:
tract = '3833'

In [ ]:
query = '\n'.join([
        'SELECT TOP 1000 ', 
        'obj.objectId, ts.ra, ts.dec, ts.redshift ', 
        'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
        'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
        'ON mt.id_truth_type=ts.id_truth_type ',
        'JOIN dp02_dc2_catalogs.Object AS obj ',
        'ON mt.match_objectId=obj.objectId ', 
        f'WHERE obj.tract={tract} '
        'AND ts.mag_r<23.0 ',
        'AND ts.truth_type=1 ',
        'AND obj.detect_isPrimary=1'])

In [ ]:
job = service.submit_job(query)
job.run()

In [ ]:
job.url

In [ ]:
job.phase

In [ ]:
%%time
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
retrieved_job = retrieve_query(job.url)
results = retrieved_job.fetch_result().to_table().to_pandas()
results

Save the results into a parquet file (or FITS, or CSV)

In [ ]:
filename = f'ztrue_tract_{tract}'
results.to_parquet(f'{filename}.parquet')

In [ ]:
plt.figure(figsize=[8,3])
plt.subplot(121)
plt.scatter(results.ra, results.dec, alpha=0.7)  
plt.xlabel('R.A. (degrees)')
plt.ylabel('Dec. (degrees)')
plt.subplot(122)
plt.hist(results.redshift)
plt.xlabel('true-z')
plt.ylabel('counts')
plt.tight_layout()

Save the notebook with current output cells (manually).

Export this notebook as HTML with the outputs up to this point to serve as description file for the sample. 

In [ ]:
cmd = f'jupyter nbconvert --to html {filename}.ipynb'
os.system(cmd)

Connect to PZ Server

In [ ]:
from pzserver import PzServer 
with open('../token.txt', 'r') as file:
    token = file.read()
pz_server = PzServer(token=token, host="pz-dev") # "pz-dev" is the temporary host during the test phase  

Prepare the upload metadata

In [ ]:
data_to_upload = {
    'name':f'ztrue tract {tract}',
    'description': f'1000 first galaxies with mag_g < 23.0 from tract {tract}', 
    'product_type': 'specz_catalog',  # Product type 
    'release': None, # LSST release, use None if not LSST data 
    'main_file': f'{filename}.parquet', # full path 
    'auxiliary_files': [f'{filename}.html']
}

In [ ]:
data_to_upload

In [ ]:
upload = pz_server.upload(**data_to_upload)  

In [ ]:
product_id = upload.product_id
product_id

In [ ]:
upload.columns

In [ ]:
column_association = {
    'objectId':'ID', 
    'ra':'RA',  
    'dec':'Dec',  
    'redshift':'z'    
}
upload.make_columns_association(column_association)

Check uploaded data product

In [ ]:
pz_server.display_product_metadata(product_id)